## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd
import os
import sys
sys.path.append(os.path.join(sys.path[0],'waveglow/'))
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

import warnings 
warnings.filterwarnings("ignore")

In [2]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower', 
                       interpolation='none')

#### Setup hparams

In [3]:
hparams = create_hparams()
hparams.sampling_rate = 22050

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



#### Load model from checkpoint

In [4]:
checkpoint_path = "tacotron2_statedict.pt"
use_cuda = True
model = load_model(hparams,use_cuda)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
if use_cuda:
    _ = model.cuda().eval().half() 
else:
    _ = model.eval()

#### Load WaveGlow for mel2audio synthesis and denoiser

In [5]:
waveglow_path = 'waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.use_cuda = use_cuda
waveglow.cuda().eval().half() if use_cuda else waveglow.eval()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow,use_cuda=use_cuda)

Infer: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 17.11it/s]


#### Prepare text input

In [6]:
text = "The blue moon, the green grass, the silent slice shook flute!"
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
device = torch.device('cuda' if use_cuda else 'cpu')
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).to(device).long()

#### Decode text input and plot results

In [7]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

#### Synthesize audio from spectrogram using WaveGlow

In [8]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

Infer: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 162.13it/s]


#### (Optional) Remove WaveGlow bias

In [9]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 

#### Long sentences

In [10]:
import textwrap

In [12]:
text = "EU leaders have reached a historic agreement on a €750bn pandemic recovery fund and their long-term spending plans following days of acrimonious debate at what was the bloc’s longest summit in two decades. As the meeting reached its fifth day, the 27 exhausted heads of state and government finally gave their seal of approval to the EU jointly borrowing debt to be disbursed through grants on an unprecedented scale in the face of an economic downturn not seen since the Great Depression."
print(len(text))
lines = textwrap.wrap(text, 180, break_long_words=False)
output = []
for line in lines:
    sequence = np.array(text_to_sequence(line, ['english_cleaners']))[None, :]
    device = torch.device('cuda' if use_cuda else 'cpu')
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).to(device).long()
    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    with torch.no_grad():
        audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
    output.append(audio[0].data.cpu().numpy())
    
outwav = np.concatenate(output)
ipd.Audio(outwav, rate=hparams.sampling_rate)

489


Infer: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 160.82it/s]

Warning! Reached max decoder steps



Infer: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 184.46it/s]
